# 02 — Bandit OPE Walkthrough

We compare IS, WIS, and Double RL on a synthetic contextual bandit. This
notebook emphasizes diagnostics: overlap, ESS, and weight tails.

## Setup

```
pip install "causalrl[plots]"
```

In [ ]:
from __future__ import annotations

from pathlib import Path

import numpy as np

from crl.benchmarks.bandit_synth import SyntheticBandit, SyntheticBanditConfig
from crl.ope import evaluate
from crl.utils.seeding import set_seed
from crl.viz import configure_notebook_display, save_figure

In [ ]:
set_seed(0)
np.random.seed(0)
configure_notebook_display()

## Run estimators

In [ ]:
benchmark = SyntheticBandit(SyntheticBanditConfig(seed=0))
dataset = benchmark.sample(num_samples=1_000, seed=1)
true_value = benchmark.true_policy_value(benchmark.target_policy)

report = evaluate(
    dataset=dataset,
    policy=benchmark.target_policy,
    estimators=["is", "wis", "double_rl"],
)
summary = report.summary_table()
summary

## Diagnostics and plots

We'll plot estimator comparisons and inspect weight distributions.

In [ ]:
fig = report.plot_estimator_comparison(truth=true_value)
fig

In [ ]:
weights = (
    benchmark.target_policy.action_prob(dataset.contexts, dataset.actions)
    / dataset.behavior_action_probs
)
fig_w = report.plot_importance_weights(weights, logy=True)
fig_w

## Save figures

These files are used in the docs site.

In [ ]:
output_dir = Path("docs/assets/figures")
output_dir.mkdir(parents=True, exist_ok=True)
save_figure(fig, output_dir / "bandit_walkthrough_estimator_comparison")
save_figure(fig_w, output_dir / "bandit_walkthrough_weights")

## Takeaways

- IS is unbiased but can be high variance.
- WIS normalizes weights to reduce variance, at the cost of bias.
- Diagnostics (ESS, overlap, tails) tell you when to trust estimates.